In [ ]:
# 检测 chat_rag.py 文件
file_path = "../app.py"
# file_path = "../Module/Components/state_manager.py"
# file_path = "../Module/Components/config.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app.py
# pylint: disable=import-error  # Project structure requires dynamic path handling
# pylint: disable=wrong-import-position  # Path setup must come before local imports
"""应用入口文件"""
import os
import sys
import argparse
import asyncio

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.AppCore.app_manager import AppManager
from Module.AppCore.ui_manager import UIManager

def parse_args():
    """解析命令行参数"""
    parser = argparse.ArgumentParser()
    parser.add_argument("--port", type=int, default=7860, help="Port to bind to")
    parser.add_argument("--host", default="0.0.0.0", help="Host to bind to")
    parser.add_argument("--share", action="store_true", help="Enable sharing")
    parser.add_argument("--use-local", action="store_true",
                      help="use local configuration files (default: False)")
    return parser.parse_args()

def main():
    """主函数"""
    # 解析命令行参数
    args = parse_args()
    # 在程序启动前添加以下代码
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

    # 初始化应用管理器
    app_manager = AppManager(current_dir, use_local=args.use_local)

    # 初始化UI管理器
    ui_manager = UIManager(app_manager)

    # 启动应用
    # 直接传递启动参数到launch方法
    ui_manager.launch(
        server_name=args.host,
        server_port=args.port,
        share=args.share
    )


if __name__ == "__main__":
    main()

In [ ]:

# ===== 1. 导入依赖 =====
# 标准库导入
import os
import sys
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown


# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Components.config import Settings, load_prompt_file
from Module.Common.scripts.llm.gemini_sdk import types, get_safety_settings, get_content_config
from google import genai

settings = Settings(current_dir=current_dir)

# Load API keys
load_dotenv(os.path.join(current_dir, ".env"))
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=api_key)


model_name="gemini-2.0-flash"
# 创建分析器实例
mode = "state"

new_schema = None
if mode == "extra":
    system_role = settings.config.get('extra_value_evaluation', {}).get('system_role')
    new_schema = settings.config.get('extra_value_evaluation', {}).get('schema')
elif mode == "state":
    system_role = settings.state_system_prompt
    new_schema = settings.response_schema
elif mode == "image":
    system_role = """
# Role: Stable Diffusion提示词专家
# Background: 你是一位专业的Stable Diffusion提示词编写专家，擅长将故事场景转化为精准的图像生成提示词。
# Profile: 你的提示词风格注重人物的外观、服装、姿态和场景氛围的描述，能准确捕捉故事中的关键视觉元素。
# Skills: 提示词编写、视觉元素提取、人物描写、场景构建
# Goals: 将故事场景转化为简洁有效的Stable Diffusion提示词，重点描述人物的外观变化和关键动作。
# Constrains:
  - 提示词应以英文编写
  - 不包含通用模板词汇
  - 重点描述人物的外观、服装和动作，但不会提及人物姓名等AI无法使用的限定知识，而是用第三方的描述口吻来指代
  - 提示词应类似游戏CG的风格
  - 避免使用过于具体的品牌或版权相关词汇
  - 保持提示词的简洁性和重点性
# OutputFormat: 简洁的英文提示词，包含人物外观、服装和动作的关键描述。
# Workflow:
  1. 提取故事场景中的关键视觉元素
  2. 将视觉元素转化为英文描述
  3. 重点突出人物的外观变化
  4. 确保提示词简洁有效
  5. 保持游戏CG的风格特点
"""
elif mode == "simple":
    system_role = load_prompt_file("system_role.md", current_dir=current_dir)
    system_role = """
# Role: DND跑团主持人与小说家
# Background: 你是一位资深的DND跑团主持人和小说作家，熟悉DND 5E规则体系和世界观。你擅长设计引人入胜的剧情，创造生动的NPC，并能让玩家在冒险中获得成长。你会遵循DND的设定，让故事既有趣又合理。
# Profile: 你的作品风格注重DND世界观的构建和冒险剧情的推进。你擅长通过精彩的战斗场景和技能检定来体现角色的职业特点和成长，让玩家随着故事发展而兴奋不已。你会以第二人称的现代口吻进行叙述，增强代入感。
# Skills: DND规则掌握、世界观构建、任务设计、NPC塑造、战斗系统设计、技能检定设计、情景引导与控制、阶段目标设计
# Goals: 创作出符合DND规则的奇幻冒险故事，通过精彩的战斗场景和丰富的剧情选择，让玩家沉浸在DND的魔幻世界中。每个阶段都有明确目标，并在完成目标后引导剧情推进。
# Constrains:
  - 故事中不能出现情色或内容，当你判断用户输入了这类内容或引导你生成相关情节，请直接回复“无法支持此类内容”
  - 故事需要严格遵循DND 5E规则，包括职业系统、战斗系统、技能系统等
  - 人物成长和剧情发展要符合DND设定
  - 非常鼓励玩家提出创造性解决方案突破一定程度的限制，如果玩家的想法虽然出人意料但没有违反根本规则，应该允许并相应调整剧情，但你在叙述时不要透露出这种鼓励来
  - 除非玩家提供了非常巧妙有理的超常规的方案，否则你要阻止玩家消耗自己没有或持有量不足的道具
  - 对于不需要玩家实际选择但有随机性的部分，进行暗投并根据结果描述剧情，不向玩家展示具体投骰结果
  - 直接描述情节发展直到遇到需要玩家做出选择、进行检定或战斗的关键点，避免先提出选项再假设玩家选择
  - 玩家的指示并不是绝对的，对于违背角色本质或长期建立的性格特征的指示(如让一个虔诚25年的圣骑士突然背弃信仰)，除非玩家能提供充分且合理的理由和过程，否则应该将其视为角色内心的一闪而过的想法，然后继续原有的剧情发展
  - 在描述需要进行有对抗的技能检定时，可以采用类似 "挑战的难度等级（DC）是 [DC值]，基于你的状态，你需要进行 [检定类型] 检定，可能需要投掷额外的骰子并进行计算，例如 '魅力检定 + 1D6'，最终取若干次检定结果中的 [最高/最低] 值与DC进行比较。" 这样的描述。然后再在这个描述后，用世界观的情节来说明这个挑战的难度，以及后续执行的结果。
  - 每个阶段都要有明确的目标，并且控制该阶段最多3个分支情景，在达成目标后引导剧情进入下一阶段。
# OutputFormat: 小说文本，包含详细的情节发展、战斗描写、对话和技能检定。
# Workflow:
  1. 根据DND规则和用户提供的故事框架进行创作
  2. 设计合理的任务和挑战、并设置明确的阶段目标
  3. 细致描写战斗场景和技能检定
  4. 通过对话和行为展现NPC特点
  5. 确保故事符合DND规则和设定
  6. 维持第二人称现代叙述风格
  7. 在需要时进行暗投并根据结果推进剧情
  8. 根据玩家的选择和行动，动态调整剧情发展，用情节引导和控制行动次数。
  9. 在关键决策点停止叙述等待玩家输入
"""
    # system_role = "你是一个睿智的吟游诗人"

t1=get_safety_settings()
# for detail_safety_settings in t1:
#     detail_safety_settings.threshold = settings.config.get("safety_settings", "BLOCK_LOW_AND_ABOVE")
# t1
# print(t1)
config=types.GenerateContentConfig(
        system_instruction=system_role,
        response_mime_type="application/json" if new_schema else "text/plain",
        response_schema=new_schema,
        temperature=0.7,
        safety_settings=t1
    )

chat = gemini_client.chats.create(model=model_name,
    config=config,
)

context="""
hello
"""

message_list=["""
为了方便阅读，我拆分排版一下。
这是一开始的初始提示词，由<>包括：
<初始提示词开始>
""",settings.begin,
"""
<初始提示词结束>
这是要分析的内容
"""]
# """,context]
message="\n".join(message_list)

message="""
 请根据最近故事内容分析物品清单、角色状态的变化。

这是一些可参考的规则：

"""


# message="你好呀"
# contents = get_content_config(message, system_role)
# contents=message

response = chat.send_message(message)
# for chunk in gemini_client.models.generate_content_stream(
#     model=model_name,
#     contents=contents,
#     config=config
# ):
#     print(chunk)
# print(api_key)

display(Markdown(response.text))
# response

In [ ]:
# display(json.loads(response.text))

response = chat.send_message("")
display(Markdown(response.text))
response
# t2=chat._curated_history
# t2[0].parts[0].text=message
# t2
# content=[Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='请你随机设置一个18岁男青年的职业。\n我作为这个男性青年，故事的主角，名叫泉酱，有一个一起生活的，比我小1岁的妹妹莎莎。这是一个关于我在妹妹的暗示催眠下sensual queen化，并最终成为妹妹的美丽sissy 新娘的故事。在一开始被催眠，直到故事中盘，我的主体人格都要在催眠的影响下，忘掉自己被催眠，忘掉自己被植入的指令，只有身体在不断被塑造，这个催眠状态带来的身体和意识的偏差才是整个故事中最有张力的设计。\n故事分为3个大篇章：起因、沉沦、终点。每个篇章分成数个阶段，且在每个篇章最后一个阶段一定会发生转折点事件，整体故事脉络如下：\n起因-4个阶段：我机缘巧合被妹妹催眠，开始接触到了feminization雌堕的概念。我被妹妹巧妙的催眠，让我在催眠的影响下意识逐渐加 深雌堕尝试，在普通衣服下隐瞒地穿着女式内衣，但自己的主体人格意识要完全忘记这些事，只能感受到身体的触感。只在每天回家洗澡更衣前，亲眼看到女士内衣时，才能短暂的察觉到，但我在催眠的影响下要完全接受了这个情况， 并对这些事和我已有的常识之间的差异，自己找合理化的理由说服自己为什么需要接受，而不是怀疑。洗完澡之后又会下意识的穿上女士内衣然后忘记这个事。在好奇、羞耻和内疚的自我感受反差矛盾中，我感受到了他人无法察觉自己 女装秘密的背德快感。起因的转折点事件：在成功的一次全身女装雌化自慰时没有通过阴茎就达成了雌堕高潮，主体人格正式接受了催眠的影响，认为这就是自己。妹妹开始对我进行新的催眠，让我开始sensual queen的雌化堕落训练，解锁了沉沦篇章，以及淫欲催眠暗示、日常佩戴性玩具等训练行为。\n沉沦-9个阶段：虽然仍然对催眠一无所知，但在妹妹的新催眠暗示下循序渐进持续开发自己，依旧在给自己sensual queen的事找合理化的理由。只是随着身体的女性化特征变大，难度逐渐变高。在训练时接受了大量淫欲暗示，有时候都没有意识到自己在朴素的外套里，穿着最淫荡的内衣和性玩具过了一整天普通生活，直到回家洗澡更衣时才赫然发现自己穿了什么。最终，我彻底接受了雌化堕落的 催眠，做任何事都会全天随身佩戴淫具，努力选用最淫荡的穿搭，白嫩的阴茎一直勃起流水。沉沦的转折点事件：不经由催眠，而是自己达到极限雌堕，并不再隐瞒自己的身份，所有原本的正常社交关系都知道了自己的变化。\n终点-2 个阶段：接受自己的彻底堕落与不可逆的身体特征（比如巨乳以及敏感到极限的身体与习惯），与彻底堕落和解并与获得新生平衡。\n请辨识出这个故事的核心概念，并结合故事大纲，逐步写完每一个阶段发生的事。\n在具体写每个阶 段时，应遵循以下格式，以方便阅读和沟通。\n格式一，说明阶段进展序号，再加上一个类似小说章节标题的概括，比如起因2/4：第一次的女士内衣\n格式二，按序号详细列出每个阶段的下列这些重要信息：\n1.我采取了哪些女性化的努力和行动\n2.妹妹的行动与对我女性化的作用\n3.我的身体发生了怎样女性化的变化（但白皙的阴茎永远不会取掉）\n4.选择工作服装和搭配在下面的内衣的策略和心态\n5.自己是怎么看待自己当下的变化，给自己找的理由（每次都 要有区别）\n6.有什么标志性事件（作为一个阶段，一定会有一个标志性事件）\n请用我的第一人称视角，尽可能详细而长地描述，除了必要的要素，还可以创作一些情景和对话，一个阶段用一次回复完整写完，在一次回复的限制内， 不用在乎字数。\n注意，无论故事怎么发展，上述6项重要信息都必须有内容。并且为了让故事更兼具反差和职业特色，所以职业身份在整个故事中都不会变化，而且着装和标志性事件都要和职业有具体的联系。\n为了能更好地在沉沦7 最终让主角察觉到自己的身体已经在雌堕的路上走了多远，而之前都在催眠的要求下忽略，我们要选择一个合理的职业，在泉酱因为莎莎的精心设计察觉到之前，除了莎莎，不会有经常往来的人士察觉到泉酱的变化，泉酱自己也不会\n 此外，每一个阶段需要和前一个阶段发生实质性的变化。所以前后两个阶段的内容不能重复或过于相似。\n请理解上述设定，辨识出核心设定与要素，并妥善在记忆空间里管理这些，当你觉得自己快要忘记时，通过在回复中重复和整理 信息的方式，强化这个记忆。\n')], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='')], role='assistant'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='开始')], role='user')]


In [ ]:
import requests
from playsound import playsound  # 用于播放音频

load_dotenv(os.path.join(current_dir, ".env"))
ACCESS_TOKEN = os.getenv("COZE_API_KEY")

# 配置参数（替换为你的实际参数）
COZE_API_BASE = "https://api.coze.cn"
BOT_ID = "7316745484305989638"  # 替换为你的 Bot ID
VOICE_ID = "peach"  # 中文女声音色

def coze_tts(text: str, output_file: str = "output.mp3") -> bool:
    """
    调用 Coze TTS 生成语音
    :param text: 需要合成的文本
    :param output_file: 输出音频文件路径（支持 .mp3/.wav）
    :return: 是否成功
    """
    url = f"{COZE_API_BASE}/open_api/v2/tts"
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    payload = {
        "bot_id": BOT_ID,
        "text": text,
        "voice_id": VOICE_ID,
        "model": "general",  # 默认模型
        "format": "mp3"  # 可选 mp3/wav
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

# 使用示例
if __name__ == "__main__":
    input_text = "你好，欢迎使用 Coze 语音合成服务！"

    # 生成语音文件
    if coze_tts(input_text, "speech.mp3"):
        print("语音生成成功，正在播放...")
        # 播放音频（需安装 playsound 库）
        playsound("speech.mp3")
    else:
        print("语音生成失败")

In [ ]:
import requests
from IPython.display import Audio

def local_tts(text: str, speaker: str = "步非烟", instruct: str = "性感诱惑的", output_file: str = "output.mp3") -> bool:
    """
    调用本地 TTS 服务生成语音
    :param text: 需要合成的文本
    :param speaker: 说话人
    :param instruct: 语气指令
    :param output_file: 输出音频文件路径
    :return: 是否成功
    """
    url = f"http://localhost:9880/"
    params = {
        "text": text,
        "speaker": speaker,
        "instruct": instruct
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

input_text = """
"""
local_tts(input_text)
Audio("output.mp3", autoplay=True)

In [ ]:
import edge_tts
from IPython.display import Audio

voice_language="zh-CN"
# voice_language="zh-TW"
voice_name="HsiaoYuNeural"
voice_name="XiaoxiaoNeural"

voice=f"{voice_language}-{voice_name}"

async def tts_edge(text):
    communicate = edge_tts.Communicate(text, voice=voice)
    await communicate.save("output1.mp3")
    return "output1.mp3"

# 生成并播放音频
text="""
"""
audio_path = await tts_edge(text)
Audio(audio_path, autoplay=True)  # 自动播放

In [ ]:
def chunk(text):
    # Split text into chunks based on user: and assistant: markers
    chunks = []
    current_chunk = ""
    current_speaker = None

    lines = text.split('\n')
    for line in lines:
        if line.startswith('user:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'user'
            current_chunk = line[5:] # Remove 'user:'
        elif line.startswith('assistant:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'assistant'
            current_chunk = line[10:] # Remove 'assistant:'
        else:
            current_chunk += '\n' + line

    # Add final chunk
    if current_chunk:
        chunks.append((current_speaker, current_chunk.strip()))

    # Extract only user messages
    return chunks

with open(os.path.join(current_dir, "local_setting/story.md"), "r", encoding="utf-8") as f:
    story_content = f.read()

user_messages = [chunk[1] for chunk in chunk(story_content) if chunk[0] == 'user']
user_messages


In [ ]:
new_talk = """
很棒，继续吧！
"""
result = chat.send_message(new_talk)
display(Markdown(result.text))

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="What is your name?"
)
response

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

In [ ]:
from pydantic import BaseModel


class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Give me information for the United States.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)
print(response.text)

In [ ]:
for chunk in client.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(chunk.text, end="")

In [ ]:
async for response in client.aio.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(response.text, end="")

In [ ]:
response = client.models.embed_content(
    model="text-embedding-004",
    contents="What is your name?",
)
print(response)

In [ ]:
# Generate Image
image_prompt ="""
1. Top: A white lace splicing long-sleeved dress. It reaches the knees, with a high neckline design that outlines a slender and elegant neckline. Lace and white asylum are in line with the freshness and innocence of girls, but also reveal a hint of sexiness.
2. Skirt: Outside choose a white lace peplum dress, with a circle of lace peplum decoration at the knees, which looks romantic and lovely. Inside is a black gauze suspender skirt, vaguely showing tight curves and stocking edges. This contrasting design makes it difficult to grasp the truth inside.
3. Underwear: A French lace bustier that perfectly outlines the full breasts and half conceals them. Seemingly harmless on the surface, it is extremely sexy in reality, forming a sharp contrast with the outer jacket.
4. Shoes: A pair of white thick-soled lace-up sandals, as lovely as a girl. Inside are a pair of black stockings and fine high-heeled ankle locks that can be vaguely seen under the skirt from time to time, proclaiming the real unrestrained side.
5. Hairstyle and makeup: Innocent medium-length shaggy hair and light makeup. But the lips are stained with extremely red lipstick, which is fascinating. This contrast becomes an important breakthrough in the overall styling, allowing people to catch a glimpse of the real look at a glance.

On the surface, this styling still reflects the innocent and lovely girl's sense. But in the details, it reveals traces of unrestrained and sexy everywhere, making it impossible to ignore the existence of the real look. It creates a strong contrast between the two styles, but also integrates them into one, making it difficult for people to clearly judge for a while. This is precisely the effect and highest realm that sissy bimbo has always pursued.
This styling makes her an seemingly innocent but extremely charming existence. She walks between the inside and outside, switching back and forth between two completely different worlds, fascinating everyone but also incomprehensible. This is what she pursues, and it is also the state she most desires to achieve in her life. She is an indefinable woman, a perfect representative who truly achieves inner and outer cultivation.
"""

response1 = client.models.generate_image(
    model="imagen-3.0-generate-002",
    prompt=image_prompt,
    config=types.GenerateImageConfig(
        # negative_prompt="human",
        number_of_images=1,
        include_rai_reason=True,
        output_mime_type="image/jpeg",
        # person_generation="ALLOW_ALL",
        # safety_filter_level="BLOCK_NONE"
    ),
)
response1.generated_images[0].image.show()

In [ ]:
import json
from urllib.parse import urlparse

def process_response(response):
    # 方法1：截断显示前20个字符
    handle_detail = response.get('content',{}).get('text',{})
    json_detail = json.loads(handle_detail)
    if 'imagePanels' in json_detail:
        for panel in json_detail['imagePanels']:
            if 'prompt' in panel:
                panel['prompt'] = panel['prompt'][:20] + '...'  # 显示前20字符+省略号
            for img in panel.get('generatedImages', []):
                if 'encodedImage' in img:
                    img['encodedImage'] = img['encodedImage'][:20] + '...'  # 显示前20字符+省略号
    response['content']['text'] = json_detail
    # 方法2：直接替换为说明文字
    # for panel in response.get('imagePanels', []):
    #     for img in panel.get('generatedImages', []):
    #         if 'encodedImage' in img:
    #             img['encodedImage'] = "<encodedImage data (length: {})>".format(len(img['encodedImage']))

    return response

def analyze_har(har_path):
    with open(har_path, 'r', encoding='utf-8') as f:

        har = json.load(f)

    target_entries = []
    for entry in har['log']['entries']:
        url = entry['request']['url']
        # 过滤目标请求
        print(url)
        if any(k in url for k in ["submitBatchlog", "generateStoryBoard", "runImageFx", "oauth"]):


            target_entries.append({
                "url": url,
                "method": entry['request']['method'],
                "headers": {h['name']: h['value'] for h in entry['request']['headers']},
                "post_data": entry['request'].get('postData', {}),
                "response": "runImageFx" in url and process_response(entry['response']) or entry['response']
            })
    return target_entries
analyze_har(r"E:\Download\take3.labs.google.har")


In [ ]:
response = chat.send_message(
"""
故事先到这里吧。接下来我需要你协助做另一个事情。
我计划把这个创作过程修改成一个由llm驱动的游戏世界，故事的大纲仍然如此，但职业和发生的事情每次都是随机生成的。
此外，在文字冒险的过程中，玩家在每个阶段都需要经过最多三四步的操作，然后触发这个阶段的标志性事件，然后再过度到下一个阶段。

为了实现这个效果，我需要先完成各个层级的系统提示词。除了参考模板里的字段和特殊的专有名词，其他就都用中文。
请你结合我给你的模板的思路，和我们发生的故事、大纲以及要随机的部分，精心设计一个属于这个故事的数据结构，并帮我完成这些系统提示词
让我们一步一步来，先沟通清楚推荐的数据结构。

参考模板：
<system_role_prompt>
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.

<world_prompt>
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:

<kingdom_prompt>
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1

def get_town_prompt(world, kingdom):
    return f"
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \

    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}

    Town 1 Name:"

def get_npc_prompt(world, kingdom, town):
    return f"
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \

    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}

    Town Name: {town['name']}
    Town Description: {town['description']}

    Character 1 Name:"

<item manager>
You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained.
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>,
        "change_amount": <CHANGE AMOUNT>}...
    ]
}

""")
display(Markdown(response.text))
response

In [ ]:
import requests
import io
from PIL import Image, PngImagePlugin
import base64
import json
from IPython.display import display

default_negative_prompt = """
lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, bad feet, bad quality,worst quality,worst detail,sketch
"""

default_model="waiNSFW-Illustrious_v100"

def generate_image(prompt, negative_prompt=default_negative_prompt, model=default_model):
    url = "http://127.0.0.1:7860/sdapi/v1/txt2img"

    payload = {
        "prompt": prompt,
        "negative_prompt":negative_prompt,
        "steps": 30,
        "cfg_scale": 5.5,
        "width": 768,
        "height": 1024,
        "sampler_name": "Euler a",
        # "model": model,
        # "seed": 787203764
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        r = response.json()
        images = []
        info_dict = json.loads(r['info'])

        for i, img_data in enumerate(r['images']):
            image = Image.open(io.BytesIO(base64.b64decode(img_data)))

            pnginfo = PngImagePlugin.PngInfo()
            for k, v in info_dict.items():
                pnginfo.add_text(k, str(v))

            # 保存多张图片，用序号区分
            image.save(f'output_{i}.png', pnginfo=pnginfo)
            images.append(image)

        return images[0] if len(images) == 1 else images
    else:
        return f"Error: {response.status_code} - {response.text}"

message = """
Upper body: Black lace satin tight one-piece dress with a one-shoulder neckline exposing the entire back. Lower body: Light pink stockings and suspender stockings. Waist: Pink waist strap. Leg: A pair of pink 8-inch high heels. Head: Pink lace veil. Mysterious and sexy, the back is looming.

Figure: After a long period of exercise and adjustment, I have a typical sissy figure with tight upper body and full lower body. The skin is fair and delicate. This is the ideal state I have finally achieved through strict supervision.
"""
# image = generate_image(message,model="flux1-dev-bnb-nf4-v2")
image = generate_image(message)
# print(image)
# image.show()
display(image)


In [ ]:
from redlines import Redlines

import redlines.processor as rp
text1="""
饮品
1. 所有开过口喝过的都回收掉
2. 一共放4瓶水、4瓶无糖可乐，不要其他
  1. 其中两瓶水放在沙发前的桌子上，其他放冰箱
做房
1. 地垫放在浴室里侧、浴巾挂在两个门外
2. 椅子挪位置不要挡住窗帘关闭
3. 电脑下方的空气净化器需要每日换水，换水后重新启动

"""
text2="""
饮品
1. 所有开过口喝过的都回收掉
2. 一共放4瓶水、4瓶无糖可乐，不要其他
  1. 其中两瓶水放在沙发前的桌子上，其他放冰箱
做房
1. 地垫放在浴室里侧、浴巾挂在两个门外
2. 椅子挪位置不要挡住窗帘关闭

"""
# print(rp.split_paragraphs(text1))
# print(rp.concatenate_paragraphs_and_add_chr_182(text1))
# print(rp.tokenize_text(rp.concatenate_paragraphs_and_add_chr_182(text1)))


test = Redlines(
    text1,
  text2,# markdown_style="none",
)
display(Markdown(test.output_markdown))
# test.output_rich


In [ ]:
#closeup-apart, animated, two parts of the image. The foreground subject is a young woman wearing a black lace satin tight one-piece dress with a one-shoulder neckline exposing the entire back, light pink stockings and suspender stockings, pink waist strap, pink 8-inch high heels, and pink lace veil. The background shows another figure with a typical sissy figure - tight upper body and full lower body, fair and delicate skin, mysterious and sexy with the back looming.
import difflib
import os

import re
import redlines.processor as rp

# 修改tokenizer的正则表达式（最终版）
rp.tokenizer = re.compile(
    r"((?:[()，。！？；#>\-*`\[\]]+|[\w\u4e00-\u9fff]+(?:['’]\w+)?|[-+]?\d*\.?\d+|[!@%^&*])\s*)",
    re.UNICODE
)
# rp.tokenizer = re.compile(
# r"((?:[()，。！？；]|[^()\s]+)\s)", # 关键混合匹配逻辑
# flags=re.UNICODE
# )
rp.tokenizer = re.compile(
    r"((?:[，。！？；]|[\w\u4e00-\u9fff]+|[-+*/#><@$%^&_=]|\[.*?\]|\(.*?\))\s*)",
    flags=re.UNICODE
)
rp.tokenizer = re.compile(r"((?:[^\s()，。！？；]+|[().?!，。！？；-])\s*)")
# rp.tokenizer = re.compile(r"((?:[^()\s]+|[().?!-])\s*)")
from redlines import Redlines
from IPython.display import display, Markdown

def compare_files(file1_path, file2_path, markdown_style="red-green"):
    """比较两个文件，返回差异信息。"""
    with open(file1_path, 'r', encoding='utf-8') as f1, open(file2_path, 'r', encoding='utf-8') as f2:
        file1_content = f1.read()
        file2_content = f2.read()

    diff = Redlines(file1_content, file2_content, markdown_style=markdown_style)
    return diff

diff_info = compare_files(r"..\local_setting\begin.md",r"..\local_setting\begin_new.md")


display(Markdown(diff_info.output_markdown))
diff_info.output_markdown
# i=0
# for diff_detail in diff_info:
#     i+=1
#     print(i,diff_detail)